In [2]:
import numpy as np
import numba

import rotations
import test_rotations

import kinematics
import test_kinematics

## Testing conversions between different representations

### Roundtrips between unit quaternions and rotation matrices

In [28]:
for i in range(1000):
    test_rotations.test_matrix_quaternion()

In [27]:
for i in range(1000):
    test_rotations.test_quaternion_matrix()

### Roundtrips between unit quaternions and Euler vectors

In [26]:
for i in range(1000):
    test_rotations.test_quaternion_euler()

In [25]:
for i in range(1000):
    test_rotations.test_euler_quaternion()
    
# Note: Only works when Euler vector has -2pi < theta < +2pi

known_euler [-0.5960656  -0.1286551   3.55911892]
euler [ 0.44110188  0.09520765 -2.63382761]


AssertionError: 

### Full roundtrips between rotation matrices and Euler vectors

In [24]:
for i in range(1000):
    test_rotations.test_matrix_euler()

In [23]:
for i in range(1000):
    test_rotations.test_euler_matrix()

known_euler [ 3.10433018  0.51909205  0.65402262]
euler [-2.96320241 -0.4954933  -0.62428972]


AssertionError: 

### Other tests

In [22]:
test_rotations.test_quaternion_from_euler()

In [21]:
for i in range(1000):
    test_rotations.test_matrix_from_euler()

In [19]:
test_kinematics.test_update_euler()

In [20]:
for i in range(1000):
    test_kinematics.test_interpolate_euler()

## Rotations greater than pi are broken!!!

In [ ]:
def roundtrip(euler):
    
    euler = np.copy(euler)
    
    rev = int(np.linalg.norm(euler) / np.pi)
    print("rev", rev)
    
    if rev > 0:
        euler /= rev*np.pi
    
    quaternion = rotations.quaternion_from_euler(euler)

    matrix = rotations.matrix_from_quaternion(quaternion)
    
    quaternion2 = rotations.quaternion_from_matrix(matrix)
    
    assert np.allclose(quaternion, quaternion2)
    
    euler2 = rotations.euler_from_quaternion(quaternion2)
    
    if rev>0:
        euler2 *= rev*np.pi
    
    return euler2

In [ ]:
def test_euler_matrix():
    while True:
        known_euler = np.random.normal(loc=0.0, scale=2*np.pi, size=3)
        if np.linalg.norm(known_euler) < 5*np.pi:
            break

    euler = roundtrip(known_euler)

    check = np.allclose(known_euler, euler)

    if not check:
        print("known_euler", known_euler)
        print("euler", euler)

    assert check

In [ ]:
test_euler_matrix()